In [1]:
#library needed to connect to S3

import s3fs
fs = s3fs.S3FileSystem()


Loading file from s3 bucket, decode and then split by line
***

In [2]:
strLoc = 's3://rogers-bucket-test-814/movie-dialog-corpus/'
fs.ls(strLoc)
with fs.open(strLoc + 'movie_lines.txt', 'rb') as f:
    fileContents = f.read()
fileContents = fileContents.decode('latin-1')
lsLines = fileContents.split('\n')

Setting up pyspark
***

In [3]:
import pyspark

We are importing pyspark and ladoing the SparkContext

In [4]:
from pyspark import SparkContext

In [5]:
sc = SparkContext('local','WordCountApp')

In [6]:
#the last line is junk so get rid of it ([0:-1])
# then parallelize the list

lineRDD = sc.parallelize(lsLines[0:-1])

In [7]:
#use map to split each line by the delimiter then get the dialouge out of there,
#finally create an item with the dialouge
wordRDD = lineRDD.map(lambda line:(line.split(' +++$+++ ')[4]))

In [8]:
print(wordRDD.collect()[0:4])

['They do not!', 'They do to!', 'I hope so.', 'She okay?']


Use map to split on spaces and then get the first word out of there
Then create an iten with the first word and a 1

In [9]:
wordRDD2 = wordRDD.map(lambda line:(line.split(' ')[0],1))

See if it worked right

In [10]:
print(wordRDD2.collect()[0:4])

[('They', 1), ('They', 1), ('I', 1), ('She', 1)]


In [11]:
#reduce by key
wordRDD2 = wordRDD2.reduceByKey(lambda x,y:x+y)

Print out and see counts

In [12]:
print(wordRDD2.collect()[0:4])

[('They', 1577), ('I', 27619), ('She', 1410), ("Let's", 752)]


In [13]:
#sort 
wordRDD2 = wordRDD2.sortBy(lambda word: -word[1])

print final sorted (first 4)

In [14]:
print(wordRDD2.collect()[0:4])

[('I', 27619), ('You', 14927), ('What', 9272), ("I'm", 6944)]


Stop the spark context

In [15]:
sc.stop()